# Predict using SageMaker Endpoint

## Model:  MobileNet (v1) SSD  300x300
## Trained For:  CFA Product Images


https://www.tensorflow.org/api_docs/python/tf/io/encode_jpeg


In [ ]:
import os, sys
import json
import numpy as np
import cv2

import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Global Variables

In [ ]:
PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/new_jpeg_images")

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564865938")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")
                                    
SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/jpeg_images/20190710_variety_1562781002.jpg"

# NAME - get this from the console
ENDPOINT_NAME = "ep-mobilenet-ssd"  

## Get a Sample Image

In [ ]:
img = tf.keras.preprocessing.image.load_img(SAMPLE_IMAGE, target_size=[300, 300])
plt.imshow(img)
plt.axis('off')

In [ ]:
x = tf.keras.preprocessing.image.img_to_array(img)
print (type(x), x.shape)

x32 = tf.keras.applications.mobilenet.preprocess_input(x[tf.newaxis,...])
print ("x32:", type(x32), x32.shape, x32.dtype)

x8 = x32.astype(np.uint8)
print ("x8:", type(x8), x8.shape, x8.dtype)

img_raw = tf.io.read_file(SAMPLE_IMAGE)
print ("tf.io.read_file:", img_raw)

img_tensor = repr(img_raw)
print ("repr:", type(img_tensor))

## Local Model

Local Model was pulled from a successful SageMaker training job (S3 -> local) and extracted.   This verifies the training job:
- created a saved_model.pb
- in export/Servo/

And, we can read the Signature Defs

In [ ]:
print ("Loading saved_model.py from:", MODEL_PATH)
loaded_model = tf.saved_model.load(sess=tf.Session(), 
                                   tags=[tf.saved_model.tag_constants.SERVING], 
                                   export_dir=MODEL_PATH)

In [ ]:
# this model complies to serving framework and can read signature defs
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve 

In [ ]:
# Signatures for:
# - serving_default
# - tensorflow/serving/predict
# appear to be the same
!saved_model_cli show --dir {MODEL_PATH} --tag_set serve --signature_def serving_default

## SageMaker Endpoint
create the endpoint assuming it doesn't already exist.  

you go to the SageMaker console
- Endpoints:  Create
- on Create & Configure
  - name:   ep-mobilenet-ssd  (whatever you want but the global name is in this code - above)
  - endpoint configuration:   use the epc-mobilenet-ssd (this specifies p2.xlarge)
  
This will take 5-10 minutes

THERE ARE MORE NOTES in the TrainModel_Step3_TrainingJob.  Creating an endpoint configuration requires knowing the inference code image (Docker?) - and I haven't figured out how to get that from the training job.

if it fails...
- retrain a model just to make sure you have a good one
- recreate the endpoint config - this seems to be the most important artifact
  

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.predictor import json_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(ENDPOINT_NAME, sagemaker_session)

print (type(predictor))

In [ ]:
# Tensor - images
image_tensor = tf.constant(value=x8, shape=(1, 300, 300, 3), dtype=tf.uint8, name="tf_example")
# image_tensor = tf.constant(value=x32, shape=(1, 300, 300, 3), dtype=tf.float32, name="tf_example")
print (image_tensor)

## You Need a tf.Example


In [ ]:
# This is needed since we cloned tensorflow/models under code.
cwd = os.getcwd()
models = os.path.join(cwd, 'code/models/research/')
slim = os.path.join(cwd, 'code/models/research/slim')
sys.path.append(models)
sys.path.append(slim)

from object_detection.inference import detection_inference

! ls {TEST_TFRECORDS_PATH}
input_tfrecord_paths = [TEST_TFRECORDS_PATH]
serialized_example_tensor, image_tensor = detection_inference.build_input( input_tfrecord_paths)

print ("serialized_example_tensor", type(serialized_example_tensor))
print ("    ", serialized_example_tensor)
print ("    ")
print ("image_tensor", image_tensor.shape)
print ("    ", image_tensor)

## Fumbling
here is where I am trying a bunch of stuff -- that doesn't   

What do we know:
- needs a list
- key must be in 'serialized_example', 'instances'
- shape of the serialized
  - list with n items generates error:  Input to reshape is a tensor with 'n' values
  - list of 1
    - error: [[{{node ParseSingleExample/ParseSingleExample}}]]\n\t [[{{node case/cond/cond_jpeg/Switch}}]]" }"
    - so it looks like it can take an Example or a jpg (on a Switch) - but, can't figure out the label for jpg

In [ ]:
# helper function to create a bytes list feature
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# read an image - encoded_jpg is type=bytes
SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/jpeg_images/20190710_variety_1562781002.jpg"
with tf.io.gfile.GFile(SAMPLE_IMAGE, 'rb') as fid:
    encoded_jpg = fid.read()
print ("encoded_jpg:", type(encoded_jpg))  # bytes
    
# tf_example = <class 'tensorflow.core.example.example_pb2.Example'>
tf_example = tf.train.Example(features=tf.train.Features(feature={'image/encoded': bytes_feature(encoded_jpg)}))
print ("tf_example:", type(tf_example))
print (" ")

tf_example1 = tf_example
tf_example2 = tf_example
# -- not using these serialization methods --

# serialize to bytes; ser_tf_example = <class 'bytes'>
#   binary string - see tutorial:  https://www.tensorflow.org/tutorials/load_data/tf_records
# ser_tf_example = tf_example.SerializeToString()
# print ("serialized to string - tf_example:", type(ser_tf_example))
# print ("    ", ser_tf_example[:20])



# serialize bytes to string
# str_ser_tf_example = ser_tf_example.decode('utf-8')
# print ("str serialized to string - tf_example:", type(str_ser_tf_example))

### Serialized list of Examples
make the list of examples THEN serialize it

#### Payload

{'instances': '[features {\n  feature {\n    key: "image/encoded"\n    value {\n      bytes_list {\n        value: "\\377\\330\\377\\340\\000\\020JFIF\\000\\001\\001\\000\\000\\001\\000\\001\\000\\000\\377\\333\\000C\\000\\002\\001\\001\\001\\001\\001\\002\\001\\001\\001\\002

#### Error

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{ "error": "JSON Value: {\n    \"instances\": {\n        \"instances\": \"[features {\\n  feature {\\n    key: \\\"image/encoded\\\"\\n    value {\\n      bytes_list {\\n        value: \\\"\\\\377\\\\330\\\\377\\\\340\\\\000\\\\020JFIF\\\\000\\\\001\\\\001\\\\000\\\\000\\\\001\\\\000\\\\001\\\\000\\\\000\\\\377\\\\333\\\\00

Excepting 'instances' to be an list/array" }". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/ep-mobilenet-ssd in account 586454201570 for more information.

In [ ]:
ser_ex1 = "{}".format(serialized_example_tensor)
ser_instance1 = {"serialized_example": ser_ex1}
print (ser_instance1)



### List of Instances (Examples)

the list of instances MUST be just 1 item.

e.g.  if you have a list of 2 items:  
ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{ "error": "Input to reshape is a tensor with 2 values, but the requested shape has 1\n\t [[{{node Reshape}}]]" }".


In [ ]:
# list of instances

ser_ex1 = "{}".format(tf_example1)
ser_instance1 = {"serialized_example": ser_ex1}
# print (ser_instance1)

ser_ex2 = "{}".format(tf_example2)
ser_instance2 = {"serialized_example": ser_ex2}
# print (ser_instance2)
ser_instance_list = [ser_instance1] #, ser_instance2, ser_instance1]

print (ser_instance_list)
predict_dict = ser_instance_list

### List of numpy arrays

No, doesn't seem to be the answer

In [ ]:
data = np.asarray(x32).tolist()
print (type(data))
ser_image = "{}".format(x32)
ser_image_list = [ser_image]
# predict_dict = {'serialized_example': ser_image_list}
predict_dict = {'instances': ser_image_list}

### After Choosing ONE:
#### Serialized List of Examples
#### List of Serialized Examples

### run this

In [ ]:
# dict to string
predict_string = json.dumps(predict_dict)
predict_json = json.loads(predict_string)

In [ ]:
print (predict_json)

In [ ]:
output_dict = predictor.predict(predict_json)

In [ ]:
data_json = json.dumps({"signature_name": "serving_default", "instances": x32})

In [ ]:
image = cv2.imread(SAMPLE_IMAGE, 1)

# resize, as our model is expecting images in 32x32.
# image = cv2.resize(image, (32, 32))
image = cv2.resize(image, (300,300))
print ("data type:", type(image), image.shape)
#image = x32

data = {'instances': np.asarray(image).astype(float).tolist()}

# The request and response format is JSON for TensorFlow Serving.
# For more information: https://www.tensorflow.org/serving/api_rest#predict_api
predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

# For more information on the predictor class.
# https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/predictor.py
predictor.predict(data)